<a href="https://colab.research.google.com/github/IgnacioMaciel/EIA/blob/main/TP2_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programacion Concurrente TP2 Parte2 MPI

Para este ejercicio se ha optado por aplicar el tema teórico **MPI** (Message Passing Interface). La finalidad del ejercicio es ampliar el conocimiento sobras la manera que posee Python para implementar la comunicación entre distintos procesos con el uso de una librería denominada **MPI4py** [1]. Para más información puede consultar la última revisión en [2].

Este ejercicio fue presentado por alumnos del curso 2020.

---
## 2.1. Ejercicio - Hola Mundo con MPI


### 2.1.1. Preguntas del ejercicio

Ejecute este ejemplo para $4$ o más instancias y responda:

**a) ¿Qué función realiza la instancia maestra? ¿Qué función realizan las instancias esclavas?**

La instancia maestra, con el ID 0, tiene la función de coordinar y controlar el flujo de trabajo. Es responsable de generar las tareas de trabajo, asignarlas a las instancias esclavas, recibir los resultados de las tareas y enviar nuevas tareas a medida que se completan. También es responsable de enviar la señal de finalización cuando todas las tareas se han completado.

Las instancias esclavas, con IDs diferentes de 0, tienen la función de realizar el trabajo asignado por la instancia maestra. Reciben las tareas del maestro, realizan el trabajo y envían los resultados.

**b) ¿Cuántas de esas instancias ejecuta la función `main()`, `initWork()` y `doWork()`?**

La función main() se ejecuta en todas las instancias, tanto en la instancia maestra como en las instancias esclavas. Esta función establece la comunicación entre los procesos y decide si un proceso es el proceso maestro o un proceso esclavo según su identificación (id).

Si el proceso es el proceso maestro (identificación id = 0), se llama a la función init() una vez para realizar ciertas inicializaciones y se generan tareas utilizando la función generateTasks(). Luego, se llama a la función initWork() pasando los parámetros necesarios.

La función initWork() se ejecuta solo en la instancia maestra. En esta función, se envían las tareas iniciales a los procesos esclavos utilizando la función comm.send(). Después de enviar las tareas iniciales, se inicia un bucle mientras haya trabajo pendiente. En este bucle, el proceso maestro recibe los resultados de los empleados (procesos esclavos) utilizando comm.recv() y envía más trabajo utilizando comm.send(). Al finalizar, se envía una señal de finalización a los procesos esclavos.

Si el proceso es un proceso esclavo (identificación id != 0), se ejecuta la función doWork(). Esta función se ejecuta solo en las instancias esclavas. En el bucle de esta función, el proceso esclavo recibe la cantidad de tiempo de descanso del proceso maestro utilizando comm.recv(). Luego, espera durante ese tiempo utilizando time.sleep() y envía el tiempo de descanso nuevamente al proceso maestro utilizando comm.send(). El proceso esclavo continúa este bucle hasta que recibe una señal de finalización.

**c) ¿Cómo se diferencian los mensajes de trabajo o de finalización?**

Los mensajes de trabajo son enviados por el maestro a las instancias esclavas utilizando el tag WORK_FLAG. Estos mensajes contienen las tareas de trabajo que las instancias esclavas deben realizar.

El mensaje de finalización es enviado por el maestro a todas las instancias (tanto maestra como esclavas) utilizando el tag END_WORK_FLAG. Este mensaje indica a las instancias que deben finalizar su trabajo y salir del bucle de trabajo.

**d) ¿Cómo implementaría la función BLAS `axpy()` con este patrón?¿Sería eficiente? *Tips*: Pide solo el planteo, no la implementación.**

Para implementar la función BLAS axpy() con este patrón, se podrían seguir los siguientes pasos:

1.   La instacia maestra genera los datos de entrada, como los vectores x e y y el escalar a.
2.   La instancia maestra divide los datos en segmentos y envía cada segmento a las instancias esclavas utilizando comm.send().
3.  Cada instancia esclava recibe su segmento de datos utilizando comm.recv().
4.  Cada instancia esclava realiza la operación axpy() en su segmento de datos.
5.  Cada instancia esclava envía su resultado parcial al maestro utilizando comm.send().
6.  La instancia maestra recibe los resultados parciales de las instancias esclavas y los combina para obtener el resultado final.

Esta implementación podría ser eficiente si los datos se dividen en segmentos de forma pareja entre las instancias esclavas y si el costo de comunicación entre la instancia maestra y las instancias esclavas no es significativo en comparación con el costo de computación de la operación axpy(). Además, como vimos en el trabajo práctico anterior, la operación axpy() es altamente paralelizable y puede aprovechar los recursos de cada instancia esclava por ende, el rendimiento puede ser bueno.

**e) ¿Qué sucede cuando solo ejecuta con una sola instancia?**

Cuando solo se ejecuta con una sola instancia, el programa se ejecutará pero no habrá instancias esclavas para realizar el trabajo. En este caso, ninguna instancia realizará las tareas.
En cambio, con dos instancias, se creará una instancia esclava que realizará todas las tareas.

**f) *Punto opcional*: El código que ejecutan las instancias esclavas, tienen un error en su lógica. ¿Cómo se podría solucionar?**

Creemos que el error se encuentra en el uso de MPI.ANY_TAG en el parámetro tag de la función comm.recv(). Esto significa que el proceso esclavo puede recibir cualquier etiqueta de mensaje, lo que incluye la etiqueta END_WORK_FLAG. Como resultado, cuando se recibe la señal de finalización, el proceso esclavo sale del bucle y finaliza su ejecución. Sin embargo, antes de finalizar, se supone que debe enviar el tiempo de descanso nuevamente al proceso maestro.

Modificando MPI.ANY_TAG por WORK_FLAG en el parámetro tag de la función comm.recv() garantizaria que el proceso esclavo solo reciba mensajes con la etiqueta WORK_FLAG y no salga del bucle hasta recibir la señal de finalización con la etiqueta END_WORK_FLAG. Además se podria agregar la funcion comm.send() con el tag END_WORK_FLAG antes del return para asegurarnos de que el tiempo de descanso se envíe al proceso maestro.

###2.1.2. Armado del ambiente

Es en donde se instalar, por única vez, el módulo MPI4py de Python en el cuaderno.

In [1]:
! pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365640 sha256=ac0ce64e25efdf6bbccc2a80cfe39dca7973bedc8d07cd1c0da3b1c44541a008
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py


### 2.1.3. Código del programa en Lenguaje Python

In [12]:
%%writefile Ejercicio2.py
from mpi4py import MPI
import numpy as np
import time

# --------------------------------------------
# Formulario
Max_tiempo_sleep =   5#@param {type: "slider", min: 1, max: 10}
Min_tiempo_sleep =   0#@param {type: "slider", min: 0, max: 10}
# --------------------------------------------

# --------------------------------------------
# Constantes de comunicacion
WORK_FLAG = 1
END_WORK_FLAG = 2
# --------------------------------------------

def main():
    comm = MPI.COMM_WORLD # Instanciamos el tipo de comunicador a utilizar.
    id = comm.Get_rank()  # Obtenemos el id como atributo del proceso que se ejecuta.

    # Utilizamos el 0 para definir al procesos Maestro, cualquier otro id sera un esclavo.
    if (id == 0) :
        init() # Llamamos funcion init para eventos que requeriremos inicialmente solo 1 vez.
        numProcesses = comm.Get_size() # Obtenemos el numero de procesos totales ejecutados.
        numTasks = (numProcesses-1)*4 # Se setea el numero de tareas.
        workTimes = generateTasks(numTasks) # Se generan las tareas, en este caso seran
        print("El jefe crea {} horas de descanso de sus empleados:".format(workTimes.size), flush=True)
        print(workTimes, flush=True)
        initWork(comm, workTimes, numProcesses)
    else:
        doWork(comm)

def generateTasks(numTasks):
    #TODO: Cambiar la semilla del random para que se generen efectivamente diferentes numeros.
    np.random.seed(1000)
    return np.random.randint(low=Min_tiempo_sleep, high=Max_tiempo_sleep, size=numTasks)

def init():
  print()
  print("Version MPI4py utilizada: {}".format(MPI.Get_version()), flush=True)
  print()
  print( "------------------------------------", flush=True)
  print( "Sistema de trabajo Suizo:", flush=True)
  print( "------------------------------------", flush=True)
  print()

def initWork(comm, workTimes, numProcesses):
    totalWork = workTimes.size
    workcount = 0
    recvcount = 0

    print("Jefe enviando las tareas iniciales:", flush=True)
    for id in range(1, numProcesses):
        if workcount < totalWork:
            work=workTimes[workcount]
            comm.send(work, dest=id, tag=WORK_FLAG) # Envia mensaje de iniciar trabajo con el dato correspondiente del array.
            workcount += 1
            print("Jefe envia trabajo y {} hs de descanso al empleado {}.".format(work, id), flush=True)
    print( "------------------------------------", flush=True)

    # Mientras haya trabajo, se recibe el resultado de los empleados y se sigue enviando MAS trabajo.
    while (workcount < totalWork) :
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat) # Recivimos resultados de los empleados.
        recvcount += 1
        workerId = stat.Get_source() # Obtenemos el identificador del empleado.
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)
        #send next work
        work=workTimes[workcount]
        comm.send(work, dest=workerId, tag=WORK_FLAG)
        workcount += 1
        print("Jefe envia nuevo trabajo y {} hs de descanso al empleado {}.".format(work, workerId), flush=True)

    while (recvcount < totalWork):
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat)
        recvcount += 1
        workerId = stat.Get_source()
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)

    for id in range(1, numProcesses):
        comm.send(0, dest=id, tag=END_WORK_FLAG)


def doWork(comm):
    while(True):
        stat = MPI.Status() # Obtiene el estado actual del empleado.
        waitTime = comm.recv(source=0, tag=MPI.ANY_TAG, status=stat) # Obtiene lo enviado por el Jefe.
        print("Soy el empleado con id {}, toca descanzo por {} hs.".format(comm.Get_rank(), waitTime), flush=True)

        if (stat.Get_tag() == END_WORK_FLAG):
            print("Marca tarjeta el empleado {}.".format(comm.Get_rank()), flush=True)
            return
        time.sleep(waitTime)
        comm.send(waitTime, dest=0)

main()


Overwriting Ejercicio2.py


### 2.1.4 Ejecución del programa

In [15]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }
NRO =   1#@param {type: "number"}
# --------------------------------------------

! mpirun --allow-run-as-root --oversubscribe -np $NRO python Ejercicio2.py


Version MPI4py utilizada: (3, 1)

------------------------------------
Sistema de trabajo Suizo:
------------------------------------

El jefe crea 0 horas de descanso de sus empleados:
[]
Jefe enviando las tareas iniciales:
------------------------------------


---
# Bibliografía

[1] MPI4py: https://mpi4py.readthedocs.io/en/stable/

[2] La versión oficial de MPI (Versión 4): https://www.mpi-forum.org/docs/mpi-4.0/mpi40-report.pdf

